# H2O_U3108 FINAL SUBMISSION

### This notebook gave us the best score on the leaderboard (MMAE 28.5644)

#### We have performed extensive analysis of the dataset, performed EDA, tried multivariate time series models like SARIMAX and also other boosting, 

#### regression-based machine learning and deep learning models using H2O 3 which can be found in **'H2O AutoML Modelling-EDA-Preprocessing Notebooks'** folder.

#### These models had MMAE scores as low as between 30 and 31. Kindly go through those notebooks as well. Thank you :)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/h2o-olympics-new-outlier-handling-scaling-std/new_train_after_outlier_removal_standardized.csv
/kaggle/input/h2o-olympics-new-outlier-handling-scaling-std/new_train_after_outlier_removal_scaled.csv
/kaggle/input/h2o-olympics-new-outlier-handling-scaling-std/new_test_after_outlier_removal_standardized.csv
/kaggle/input/h2o-olympics-new-outlier-handling-scaling-std/new_test_after_outlier_removal.csv
/kaggle/input/h2o-olympics-new-outlier-handling-scaling-std/new_test_after_outlier_removal_scaled.csv
/kaggle/input/h2o-olympics-new-outlier-handling-scaling-std/new_train_after_outlier_removal.csv
/kaggle/input/h2o-olympics-h2o-world-india/AQI_Hackathon/sample_submission.csv
/kaggle/input/h2o-olympics-h2o-world-india/AQI_Hackathon/train.csv


# Installations

In [ ]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.8 MB/s eta 0:00:0000:01


# Imports

In [ ]:
# Import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import pmdarima as pm

warnings.filterwarnings('ignore')
%matplotlib inline

tqdm.pandas()

# Reading sample submission, training, testing(validation) files

In [ ]:
submission_df = pd.read_csv('/kaggle/input/h2o-olympics-h2o-world-india/AQI_Hackathon/sample_submission.csv')
submission_df

,ID_Date,AQI
0,AS_4_2022-12-04,10.0
1,AS_4_2022-12-05,10.0
2,AS_4_2022-12-06,10.0
3,AS_4_2022-12-07,10.0
4,AS_4_2022-12-08,10.0
...,...,...
1115,WB_16_2022-12-27,10.0
1116,WB_16_2022-12-28,10.0
1117,WB_16_2022-12-29,10.0
1118,WB_16_2022-12-30,10.0


In [ ]:
base_input_path = '/kaggle/input/h2o-olympics-new-outlier-handling-scaling-std/'

final_train_df_scaled = pd.read_csv(base_input_path+'new_train_after_outlier_removal_scaled.csv',index_col=False)
final_test_df_scaled = pd.read_csv(base_input_path+'new_test_after_outlier_removal_scaled.csv',index_col=False)

final_train_df_standardized = pd.read_csv(base_input_path+'new_train_after_outlier_removal_standardized.csv',index_col=False)
final_test_df_standardized = pd.read_csv(base_input_path+'new_test_after_outlier_removal_standardized.csv',index_col=False)

#### The scaled (MinMax scaled) and standardized (Standard scaled) data are the scaled/standardized versions of the data we obtained upon training Univariate Auto ARIMA
#### on each pollutant variable and other custom created feature variables per location to get the future values of those variables.

#### In the case of Univariate Auto ARIMA on just the AQI column, here, we are using the scaled dataset which was obtained after EDA and multiple preprocessing steps:

#### - Feature Extraction
 #### - Null value handling by imputation and interpolation
 #### - Conversion of categorical columns to numerical columns 
####  - Missing Dates Handling
 #### - Outlier detection and handling for all relevant features as per each location
 #### - Pollutant subindex and AQI formula feature addition


In [ ]:
final_test_df_scaled

,StateCode,Date,PM2.5,PM10,O3,CO,SO2,DayOfWeek,DayOfMonth,WeekOfYear,Month,DayOfYear,ID,ID_Date
0,AS,2022-12-04,0.051115,0.043161,0.256249,0.111112,0.337956,6,4,48,12,338,0,AS_4_2022-12-04
1,AS,2022-12-05,0.053779,0.047102,0.243449,0.115805,0.336496,0,5,49,12,339,0,AS_4_2022-12-05
2,AS,2022-12-06,0.055170,0.047102,0.236738,0.117611,0.335595,1,6,49,12,340,0,AS_4_2022-12-06
3,AS,2022-12-07,0.055170,0.047102,0.233220,0.117611,0.335040,2,7,49,12,341,0,AS_4_2022-12-07
4,AS,2022-12-08,0.055170,0.047102,0.231376,0.117611,0.334697,3,8,49,12,342,0,AS_4_2022-12-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,WB,2022-12-27,0.044381,0.041856,0.203705,0.163963,0.172365,1,27,52,12,361,39,WB_16_2022-12-27
1116,WB,2022-12-28,0.044381,0.041856,0.203708,0.163963,0.172296,2,28,52,12,362,39,WB_16_2022-12-28
1117,WB,2022-12-29,0.044381,0.041856,0.203710,0.163963,0.172230,3,29,52,12,363,39,WB_16_2022-12-29
1118,WB,2022-12-30,0.044381,0.041856,0.203713,0.163963,0.172167,4,30,52,12,364,39,WB_16_2022-12-30


In [ ]:
final_train_df_scaled

,StateCode,Date,PM2.5,PM10,O3,CO,SO2,AQI,DayOfWeek,DayOfMonth,WeekOfYear,Month,DayOfYear,ID
0,AS,2021-08-17,0.141336,0.161869,0.107248,0.123800,0.298256,94.0,1,17,33,8,229,0
1,AS,2021-08-18,0.162247,0.117044,0.205739,0.121221,0.307617,124.0,2,18,33,8,230,0
2,AS,2021-08-19,0.166630,0.106943,0.250663,0.131538,0.349101,82.0,3,19,33,8,231,0
3,AS,2021-08-20,0.154709,0.126622,0.272942,0.126380,0.486639,107.0,4,20,33,8,232,0
4,AS,2021-08-21,0.233020,0.183980,0.265637,0.281130,0.486639,107.0,5,21,33,8,233,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25818,WB,2022-11-29,0.044144,0.045366,0.217627,0.164612,0.073802,47.0,1,29,48,11,333,39
25819,WB,2022-11-30,0.041299,0.040005,0.221826,0.160630,0.061100,52.0,2,30,48,11,334,39
25820,WB,2022-12-01,0.038051,0.041646,0.161920,0.169922,0.113003,41.0,3,1,48,12,335,39
25821,WB,2022-12-02,0.048340,0.040839,0.179027,0.151337,0.154612,40.0,4,2,48,12,336,39


# Using Univariate Auto ARIMA individually for each of the 40 locations

In [ ]:
train = final_train_df_scaled.copy()
sample_df = final_test_df_scaled[['Date', 'ID']].copy()
df = pd.DataFrame(columns = sample_df.columns)  

for station in train['ID'].unique():
    print('NEW LOCATION -',station)
    location_train_df = train[train["ID"] == station]
    location_test_df = sample_df[sample_df["ID"] == station]
    print(location_train_df.shape)
#     fig = px.line(location_train_df, x='Date', y='AQI', title='AQI')
#     fig.update_xaxes(
#     rangeslider_visible=True,
#     rangeselector=dict(
#         buttons=list([
#             dict(step="all")])))
#     fig.show()
    el_df=location_train_df.set_index('Date')
#     el_df.plot(subplots=True)
    final_df = el_df
    model = pm.auto_arima(final_df['AQI'],test='adf',error_action='ignore',  
                           suppress_warnings=True,
                      stepwise=True, trace=True, maxiter=1000, n_jobs=-1)
    model.fit(final_df['AQI'])
    forecast=model.predict(n_periods=28, return_conf_int=True)
    print(len(forecast[0]))
    print(forecast[0])
    location_test_df['AQI'] = np.array(forecast[0])
    print(location_test_df)
    df = df.append(location_test_df,ignore_index = True)
    print(df)

NEW LOCATION - 0
(474, 14)
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=5176.183, Time=0.66 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=5261.152, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=5255.030, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=5239.811, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=5259.152, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=5174.212, Time=0.25 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=5176.644, Time=0.16 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=5184.934, Time=0.15 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=5172.309, Time=0.45 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=5173.755, Time=0.27 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=5174.294, Time=0.65 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=5174.299, Time=0.72 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=5175.094, Time=0.54 sec
 ARIMA(2,1,4)(0,0,0)[0] intercept   : AIC=5175.703, Time=1.27 sec
 ARIMA

# Changing sample submission AQI column to the predicted AQI values

In [ ]:
submission_df['AQI'] = df['AQI']
submission_df.to_csv('submission_new_auto_arima_univ_after_outlier_handling.csv', index=False)

In [ ]:
submission_df

,ID_Date,AQI
0,AS_4_2022-12-04,57.686524
1,AS_4_2022-12-05,56.346240
2,AS_4_2022-12-06,54.357411
3,AS_4_2022-12-07,55.639851
4,AS_4_2022-12-08,54.812906
...,...,...
1115,WB_16_2022-12-27,48.472447
1116,WB_16_2022-12-28,48.472447
1117,WB_16_2022-12-29,48.472447
1118,WB_16_2022-12-30,48.472447


# Based on previous runs of ARIMA, it was observed that it over-estimates its predictions. To handle this, we took 97.5% of each of our predictions obtained above.

In [ ]:
submission_df['AQI'] = df['AQI'].apply(lambda x: x*0.975)
submission_df.to_csv('submission.csv', index=False)

#### As mentioned before please go through the notebooks in the **'H2O AutoML Modelling-EDA-Preprocessing Notebooks'** folder.

#### Thank you :)